## Load the credit card transactions data

In [1]:
import pandas as pd

In [11]:
data = pd.read_csv('card transactions_cleaned.csv')

In [3]:
data.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
0,1,5142190439,1/1/10,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
1,2,5142183973,1/1/10,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0
2,3,5142131721,1/1/10,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0
3,4,5142148452,1/1/10,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0
4,5,5142190439,1/1/10,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0


## 1. Remove all the transactions from FedEx

In [12]:
df = data[data['Merch description'].str.contains('FEDEX')==False]
df.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
1,2,5142183973,1/1/10,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0
2,3,5142131721,1/1/10,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0
7,8,5142191182,1/1/10,6098208200062,MIAMI COMPUTER SUPPLY,OH,45429.0,P,230.32,0
8,9,5142258629,1/1/10,602608969534,FISHER SCI ATL,GA,30091.0,P,62.11,0
13,14,5142124791,1/1/10,5725000466504,CDW*GOVERNMENT INC,IL,60061.0,P,106.89,0


## 2. Derive the first digit of amount

In [24]:
df['Amount'].min()

0.01

In [40]:
def firstindecimal(x):
    while x < 1:
        x *= 10
    return str(x)[0]

In [41]:
df['digit']=df['Amount'].apply(lambda x: str(x)[0] if x>=1 else firstindecimal(x))
df.head()

/Users/danyiwang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud,digit
1,2,5142183973,1/1/10,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0,3
2,3,5142131721,1/1/10,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0,1
7,8,5142191182,1/1/10,6098208200062,MIAMI COMPUTER SUPPLY,OH,45429.0,P,230.32,0,2
8,9,5142258629,1/1/10,602608969534,FISHER SCI ATL,GA,30091.0,P,62.11,0,6
13,14,5142124791,1/1/10,5725000466504,CDW*GOVERNMENT INC,IL,60061.0,P,106.89,0,1


In [43]:
# check if it check the correct digit for fractions
df[df['Amount']<1][['Amount','digit']].tail()

,Amount,digit
90831,0.01,1
91028,0.99,9
94261,0.72,7
94461,0.78,7
94966,0.30,3


## 3. Group by Card# and Merch# -> distribution

In [45]:
df_card = df[['Recnum','Cardnum','digit']].groupby(['Cardnum','digit']).count().reset_index()
df_card.head()

,Cardnum,digit,Recnum
0,5142110002,1,1
1,5142110081,4,2
2,5142110081,6,2
3,5142110313,1,2
4,5142110313,4,1


In [49]:
df_merch = df[['Recnum','Merchnum','digit']].groupby(['Merchnum','digit']).count().reset_index()
df_merch.head(20)


,Merchnum,digit,Recnum
0,0,1,20
1,0,2,7
2,0,3,6
3,0,4,7
4,0,5,6
5,0,6,3
6,0,8,3
7,0,9,1
8,003100006NOT6,5,1
9,004740006ABC6,2,1


## 4. For cardnum

In [54]:
df_card['low']=df_card['digit'].apply(
    lambda x: 1 if x=='1' or x=='2' else 0)

,Cardnum,digit,Recnum,low
0,5142110002,1,1,1
1,5142110081,4,2,0
2,5142110081,6,2,0
3,5142110313,1,2,1
4,5142110313,4,1,0
5,5142110402,1,1,1
6,5142110402,2,2,1
7,5142110402,3,2,0
8,5142110402,4,2,0
9,5142110402,5,4,0


In [87]:
df_card_n = df_card.pivot_table(index = 'Cardnum', 
                                     values = 'Recnum', 
                                     columns = 'low',
                                     aggfunc = sum).reset_index()
df_card_n.rename(columns={0:'n_high',1:'n_low'},inplace=True)
df_card_n.head()

low,Cardnum,n_high,n_low
0,5142110002,NaN,1.0
1,5142110081,4.0,NaN
2,5142110313,1.0,2.0
3,5142110402,8.0,3.0
4,5142110434,NaN,1.0


In [88]:
df_card_n = df_card_n.apply(lambda s:s.fillna(1)) # avoid divided by zero
df_card_n['n']=df_card_n['n_high']+df_card_n['n_low']
df_card_n.head()

low,Cardnum,n_high,n_low,n
0,5142110002,1.0,1.0,2.0
1,5142110081,4.0,1.0,5.0
2,5142110313,1.0,2.0,3.0
3,5142110402,8.0,3.0,11.0
4,5142110434,1.0,1.0,2.0


In [130]:
def unusual_base(nlow,nhigh):
    R = 1.096*nlow/nhigh
    return max(R,1/R)
def unusual_smooth(nlow,nhigh):
    from math import exp
    n = nlow + nhigh
    r = 1.096*nlow/nhigh
    t = (n-15)/3
    R = 1+(r-1)/(1+exp(-t))
    return max(R,1/R)

In [131]:
df_card_n['unusual'] = df_card_n.apply(
    lambda x: unusual_smooth(x['n_low'],x['n_high']),
    axis = 1
)

In [132]:
top40card = df_card_n.sort_values(by='unusual',ascending=False).head(40).reset_index(drop=True)
top40card.head()

low,Cardnum,n_high,n_low,n,unusual
0,5142253356,5.0,61.0,66.0,13.371199
1,5142299705,3.0,25.0,28.0,9.027976
2,5142197563,134.0,15.0,149.0,8.150852
3,5142194617,33.0,5.0,38.0,6.007779
4,5142239140,3.0,16.0,19.0,4.834555


## 5. For Merchnum

In [133]:
df_merch['low']=df_merch['digit'].apply(
    lambda x: 1 if x=='1' or x=='2' else 0)

df_merch_n = df_merch.pivot_table(index = 'Merchnum', 
                                     values = 'Recnum', 
                                     columns = 'low',
                                     aggfunc = sum).reset_index()
df_merch_n.rename(columns={0:'n_high',1:'n_low'},inplace=True)

df_merch_n = df_merch_n.apply(lambda s:s.fillna(1)) # avoid divided by zero
df_merch_n['n']=df_merch_n['n_high']+df_merch_n['n_low']

df_merch_n['unusual'] = df_merch_n.apply(
    lambda x: unusual_smooth(x['n_low'],x['n_high']),
    axis = 1
)

In [134]:
top40merch = df_merch_n.sort_values(by='unusual',ascending=False).head(40).reset_index(drop=True)
top40merch.head()


low,Merchnum,n_high,n_low,n,unusual
0,991808369338,181.0,1.0,182.0,165.145985
1,8078200641472,1.0,59.0,60.0,64.663981
2,308904389335,53.0,1.0,54.0,48.352488
3,3523000628102,1.0,34.0,35.0,37.217908
4,808998385332,36.0,1.0,37.0,32.177589


In [135]:
top40card.to_csv('card40.csv')
top40merch.to_csv('merch40.csv')